In [1]:
import numpy as np
import pandas as pd

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import pickle
import sys

from sklearn import metrics
from sklearn.svm import SVC

def get_probs_positive_class(pred):
        positive_probs = []

        for prediction in pred:
            positive_probs.append(prediction[1])
        return positive_probs
    
    
dataset_path = "/home/colombelli/Documents/research_proc_ds/BRCA/processedBRCA.rds"
read_RDS = robjects.r['readRDS']
df = read_RDS(dataset_path)

with localconverter(robjects.default_converter + pandas2ri.converter):
    pydf = robjects.conversion.rpy2py(df)

    
fold_path = "/home/colombelli/Documents/brca_new_sampling/fold_sampling.pkl"
with open(fold_path, 'rb') as file:
    folds_sampling = pickle.load(file)

In [4]:
fold2 = folds_sampling[1]
training = pydf.loc[fold2[0]]
testing = pydf.loc[fold2[1]]

ranking_output_path = "/home/colombelli/Desktop/BRCA_no_IQR/fold_1/agg_ranking.rds"

read_RDS = robjects.r['readRDS']
ranking = read_RDS(ranking_output_path)

with localconverter(robjects.default_converter + pandas2ri.converter):
    pyrnk = robjects.conversion.rpy2py(ranking)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

best_genes = list(pyrnk.index.values)
best_genes = best_genes[0:5]
print("Selected Genes:", best_genes)

training_x = training.loc[:, best_genes]
testing_x = testing.loc[:, best_genes]

training_y = training.loc[:, ['class']].T.values[0]
testing_y = testing.loc[:, ['class']].T.values[0]

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(training_x, training_y)

print("Accuracy:", clf.score(testing_x, testing_y))
pred = clf.predict_proba(testing_x)
pred = get_probs_positive_class(pred)
print("AUC score:", metrics.roc_auc_score(np.array(testing_y, dtype=int), pred))

Selected Genes: ['MIR4508', 'FIGF', 'IBSP', 'CA4', 'LRRC3B']
Accuracy: 0.9919028340080972
AUC score: 1.0


In [6]:
from sklearn.metrics import confusion_matrix

pred = clf.predict(testing_x)

In [7]:
pred

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1.

In [8]:
cm = confusion_matrix(testing_y, pred)

In [9]:
cm

array([[ 23,   0],
       [  2, 222]])

In [10]:
cm.ravel()

array([ 23,   0,   2, 222])

In [11]:
type(cm)

numpy.ndarray

In [12]:
import pickle

file = "/home/colombelli/Documents/ExperimentsNewTHs/KIRP/test/final_confusion_matrices.pkl"

with open(file, 'rb') as handle:
    conf_matrices = pickle.load(handle)

In [17]:
conf_matrices[0][15]

array([[7, 0],
       [1, 5]])

In [18]:
conf_matrices[0][15].ravel()

array([7, 0, 1, 5])